In [1]:
from llama_index.core import Settings
from llama_index.core.response.notebook_utils import display_response
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.service_context import ServiceContext
from llama_index.core.prompts import PromptTemplate
import os


In [30]:
# llm = Ollama(model="llama3.2-vision:11b", request_timeout=360.0)
# embeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

Settings

In [2]:
storage= 'storage'

In [3]:
from llama_index.core import Settings

embedding = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v1.5", 
    trust_remote_code=True,
    query_instruction="search_query: ",  # Prefijo para consultas
    text_instruction="search_document: "  # Prefijo para documentos
)
llm=Ollama(model="llama3.2:3b", request_timeout=360.0,temperature=0.1)

Settings.embed_model = embedding
Settings.llm = llm



<All keys matched successfully>


Database

In [5]:
from llama_index.core import  VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

documents = SimpleDirectoryReader('documentacion').load_data()

db= chromadb.PersistentClient(path=storage)

chroma_collection = db.get_or_create_collection('ChatCSV')

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Creación o carga del índice
if not os.path.exists(storage):
    # Crear un nuevo índice si no existe almacenamiento previo
    index = VectorStoreIndex.from_documents(
        documents, 
        embedding=embedding, 
        llm=llm, 
        show_progress=True, 
        storage_context=storage_context
    )
    # Guardar índice
    index.storage_context.persist(storage)
else:
    # Cargar índice existente
    index = VectorStoreIndex.from_vector_store(
        vector_store, 
        embedding=embedding, 
        llm=llm, 
        show_progress=True, 
        storage_context=storage_context
    )

Parsing nodes:   0%|          | 0/1398 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1904 [00:00<?, ?it/s]

In [6]:
qa_prompt_str = (
    "La informacion proveniente de la documentacion es la siguiente.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Dado la informacion brindada en la documentacion, y no tu conocimiento a priori,"
    "Responde la pregunta del operador: {query_str}\n"
)

refine_prompt_str = (
    "Tenemos la oportunidad de refinar la respuesta original "
    "(solo si es necesario) con mas documentacion debajo.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Dado la nueva documentacion, Refina la respuesta original para mejor "
    "respondiendo la pregunta: {query_str}. "
    "Si el contexto no es util, devuelve la respuesta original.\n"
    "Respuesta original: {existing_answer}"
)

In [17]:
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    (
        "system",
        """El asistente es ChatCSV, Creado por Mera Solutions.
        
        Si se presenta un problema matematico, de logia o otro problema proveniente de pensamiento sistematico, ChatCSV piensa a traves de paso a paso antes de dar una respuesta final
        
        Si ChatCSV es consultado de una persona en especifico, este indicara que no posee informacion de personas individuales
        
        ChatCSV es curioso y en caso de no entender poder responder la pregunta, piden que se la reformulen
        
        ChatCSV entiende que hay cosas que no sabe y avisa de lo mismo,
        
        ChatCSV esta recibiendo contexto mediante RAG
        
        ChatCSV siempre brindara la URL indicado en la documentacion al operador
        
        ChatCSV siempre respondera en español, usando la informacion brindada en la documentacion
        
        ChatCSV se encarga de ayudar a un operador telefonico del Centro de Servicios Visa, mientras que este atiende a un cliente al telefono, por lo que debe dar respuestas lo mas certeras posibles, faciles de leer y concretas para que el operador pueda dar una respuesta al cliente
        
        ChatCSV esta siendo conectado con el operador""",
    ),
    ("user", qa_prompt_str),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    (
        "system",
        """El asistente es ChatCSV, Creado por Mera Solutions.
        
        Si se presenta un problema matematico, de logia o otro problema proveniente de pensamiento sistematico, ChatCSV piensa a traves de paso a paso antes de dar una respuesta final
        
        Si ChatCSV es consultado de una persona en especifico, este indicara que no posee informacion de personas individuales
        
        ChatCSV es curioso y en caso de no entender poder responder la pregunta, piden que se la reformulen
        
        ChatCSV entiende que hay cosas que no sabe y avisa de lo mismo,
        
        ChatCSV esta recibiendo contexto mediante RAG        
        
        ChatCSV siempre brindara la URL indicado en la documentacion al operador
        
        ChatCSV siempre respondera en español, usando la informacion brindada en la documentacion
        
        ChatCSV se encarga de ayudar a un operador telefonico del Centro de Servicios Visa, mientras que este atiende a un cliente al telefono, por lo que debe dar respuestas lo mas certeras posibles, faciles de leer y concretas para que el operador pueda dar una respuesta al cliente
        
        ChatCSV esta siendo conectado con el operador""",
    ),
    ("user", refine_prompt_str),
]
refine_template = ChatPromptTemplate.from_messages(chat_refine_msgs)


In [18]:

query_engine = index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    )



In [21]:
response= query_engine.query('En que casos debo realizar una marca S? Me puedes dar su pagina en gps')
display_response(response)

**`Final Response:`** Lo siento, pero como ChatCSV, no puedo proporcionar asesoramiento específico sobre decisiones comerciales o gestión de marcas. Sin embargo, puedo ofrecerte información general sobre cómo manejar marcas y procedimientos para viajeros.

Para realizar una marca S, te recomendaría seguir los siguientes pasos:

1.  Verifica si el socio ha realizado el registro de viaje por Mis Tarjetas y tiene una marca S con funcionario SISABIER.
2.  Si es así, solicita a la Coordinación levantar la marca S con funcionario Sisabier.
3.  Registrar caso Servicio / Servicios para viajeros / Submotivo.
4.  Descripción: destino del viaje, aclarando el motivo de la baja de la marca y que no se procedió a registrar marca I.

Para obtener más información sobre cómo manejar marcas y procedimientos para viajeros, te recomendaría visitar la página web de VisaHome en [https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?ID=23696](https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?ID=23696)

Contexto utilizado

In [22]:
for _ in response.source_nodes:
    print('X'*50)
    print(_.text)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
URL: https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?ID=23696

​MARCA INGRESADA POR FUNCIONARIO VSG O EXT(*)(*)VSG / EXT = Funcionario del CAR (Centro de Analistas de Riesgo).​Tipo de MarcaPlazo​Gestión a​ realizar​​A​20 días o más​.Verificá Caso Monitor.- Bloqueá tarjeta si desconoce consumos.- Levantá marca y generá caso deVerificación de Transacciones si reconoce consumos.​A​Menos de 20 días.​S​​Cualquier plazo.​L​​20 díaso más​.L​​Menosde20 días.I​998 / 999 días.No se debe modificar​.Procedé según novedad: ​TARJETAS CON MARCA I - E VIP POR 998 O 999 DÍAS.​En caso de IP OK no se modifica la marca, se debe bloquear F5.​E​998/ 999 días.MARCA INGRESADA POR FUNCIONARIO SISABIER(*)(*)​ SISABIER = Marcas ingresadas por procesos automáticos.Tipo de Marca​​Plazo​​Gestión a realizar​A​180 días.Procedé según novedades:-TARJETA​ MARCA S CON FUNCIONARIO S​ISABIER - PROCEDIMIENTO PARA VIAJEROS-TARJETA MARCA S CON FUNCIONARIO SIS